<a href="https://colab.research.google.com/github/Babitha23/Cyberbullying-detection/blob/main/data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New Section

In [ ]:
import pandas as pd
import numpy as np
import re
#load data from csv file and concat
data1 = pd.read_csv("sessions_0plus_to_10_metadata.csv", index_col=0, encoding = "ISO-8859-1")
data2 = pd.read_csv("sessions_10plus_to_40_metadata.csv", index_col=0, encoding = "ISO-8859-1")
data3 = pd.read_csv("sessions_40plus_metadata.csv", index_col=0, encoding = "ISO-8859-1")
data = pd.concat([data1,data2,data3], axis=0)

In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
#check shape of the dataframes
print(data1.shape)
print(data2.shape)
print(data3.shape)
print(data.shape)

(278, 214)
(1018, 214)
(922, 214)
(2218, 214)


In [ ]:
#Drop unwanted columns
data = data.drop(['_golden', '_unit_state', '_trusted_judgments', '_last_judgment_at', 'question1:confidence', 'question2:confidence', 'cptn_time', 'img_url', 'id', 'owner_id', 'cyberaggression', 'cyberbullying'], axis=1)

#Replace empty comments  with null value
data = data.replace('empety', np.NaN)
data = data.replace('empty', np.NaN)

#Create new column with concat of all the comments excluding null values
data['comments'] = data.iloc[:, 2:197].fillna('').agg('.'.join, axis=1)

#Remove individual comments columns
data = data.drop(data.iloc[:, 2:197], axis=1)

#concat owner's post along with comments
data['sessiontext'] = data['owner_cmnt'] + '.' + data['comments']

#Dropping owner_cmnt and comments columns
data = data.drop(['owner_cmnt', 'comments'], axis=1)

data.head()

,question1,question2,likes,shared media,followed_by,follows,sessiontext
_unit_id,,,,,,,
702714440,noneAgg,noneBll,4313\r\r likes,1477,53913,268,gosh we love these five idiots so much __ for...
702714441,noneAgg,noneBll,2102\r\r likes,761,426533,379,Anyone see the Pepsi halftime show commercial ...
702714442,noneAgg,noneBll,108939\r\r likes,698,1974089,389,Today I ate cr_pes and bought a drawing of ch...
702714443,aggression,bullying,6441\r\r likes,1257,349594,656,New works in progress for an upcoming show at ...
702714444,noneAgg,noneBll,1153\r\r likes,2052,60772,388,"Due to overwhelming response, we have went ahe..."


In [ ]:
def cleancomments(text):
    pattern1 = re.compile('\(created.at:\d\d\d\d-\d\d-\d\d \d\d:\d\d:\d\d\)')
    pattern2 = re.compile('<font color\=\"\#0066CC\"\>.*?\<\/font>')
    pattern3 = re.compile('@[^\s]+')
    temp = re.sub(pattern1, '', str(text))
    temp1 = re.sub(pattern2, '', str(temp))
    temp2 = re.sub(pattern3, '', str(temp1))
    return temp2

def cleanlike(text):
    pattern = re.compile('[0-9]+')
    num = re.findall(pattern, text)
    return num[0]

def checkcomments():
    count = 0
    count1 = 0
    count2 = 0
    for i in range(2218):
        count = count + len(re.findall('\(created.at:\d\d\d\d-\d\d-\d\d \d\d:\d\d:\d\d\)', str(data.iloc[i, :]['sessiontext'])))
        count1 = count1 + len(re.findall('<font color\=\"\#0066CC\"\>.*?\<\/font>', str(data.iloc[i, :]['sessiontext'])))
        count2 = count2 + len(re.findall('@[^\s]+', str(data.iloc[i, :]['sessiontext'])))
    print("time pattern appears", count, "times")
    print("html pattern appears", count1, "times")
    print("User tagging in owner's comment appear", count2, "times")

def cleaning(text):
    """ Removes unicode strings like "\u002c" and "x96" """
    tmp = re.sub(r'(\\u[0-9A-Fa-f]+)',r'', text)       
    temp = re.sub(r'[^\x00-\x7f]',r'',tmp)
    #removing url
    temp1 = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','url',temp)
    temp2 = re.sub(r'#([^\s]+)', r'\1', temp1)
    temp3 = re.sub(r'#([^\s]+)', r'\1', temp2)
    #removing emoticons
    temp4 = re.sub(':\)|;\)|:-\)|<3|\(-:|:-D|=D|:P|xD|X-p|\^\^|:-*|\^\.\^|\^\-\^|\^\_\^|\,-\)|\)-:|:\'\(|:\(|:-\(|:\S|T\.T|\.\_\.|:<|:-\S|:-<|\*\-\*|:O|=O|=\-O|O\.o|XO|O\_O|:-\@|=/|:/|X\-\(|>\.<|>=\(|D:', '', temp3)
    temp5 = re.sub(' +', ' ', temp4) #removing additional whitespaces
    temp6 = re.sub(r"(\.)\1+", '.', temp5)
    temp7 = re.sub(r"(\!)\1+", '!', temp6)
    return temp7

In [ ]:
checkcomments()

time pattern appears 156357 times
html pattern appears 235463 times
User tagging in owner's comment appear 70493 times


In [ ]:
#Removing html tags, usernames, comment time and cleaning 'likes' by calling the functions
for i in range(2218):
    data['sessiontext'] = data['sessiontext'].apply(lambda x: cleancomments(x))
    data['likes'] = data['likes'].apply(lambda y: cleanlike(y))
    
print("After cleaning:")
print("---------------")
checkcomments()

After cleaning:
---------------
time pattern appears 0 times
html pattern appears 0 times
User tagging in owner's comment appear 0 times


In [ ]:
for i in range(2218):
    data['sessiontext'] = data['sessiontext'].apply(lambda x: cleaning(x))

In [ ]:
#data.to_csv('cleandata.csv')

In [ ]:
#data = pd.read_csv("/cleandata.csv", index_col=0, encoding = "ISO-8859-1")
#data.iloc[2211, :]['sessiontext']